In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.regularizers import L2
from sklearn.model_selection import TimeSeriesSplit
from keras.layers import LSTM, Dense, Dropout, BatchNormalization , Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go

In [ ]:
# خواندن داده و فیلتر کردن از تاریخ ۲۰۱۸ به بعد
df = pd.read_csv('BTC_USD_INDICATORs.csv')

# حذف ستون 'Adj Close' و تنظیم ستون Date به عنوان اندیس
df = df.drop('Adj Close', axis=1)
df['Volume'] = np.log1p(df['Volume'])
df.head(-10)

In [ ]:
# Check shape
df.shape

In [ ]:
df.describe()

In [ ]:
# Plot historical closing prices
plt.close()
plt.figure(1, figsize=(12, 6))
plt.plot(df["Close"])
plt.show()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])

# رسم نمودار کندلی
fig = go.Figure(data=[go.Candlestick(x=df['Date'],
                                     open=df['Open'], high=df['High'],
                                     low=df['Low'], close=df['Close'])])

fig.update_layout(title='نمودار کندلی',
                  xaxis_title='تاریخ',
                  yaxis_title='قیمت')

fig.show()

In [ ]:
df = df.set_index('Date')
features = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA_20', 'MA_50', 'MA_200', 'RSI', 'MACD', 'Signal', 'Histogram']
x = df[features].values  # فقط مقادیر عددی ستون‌های موردنظر

scaler = MinMaxScaler(feature_range=(0, 1))
x = scaler.fit_transform(np.array(x))

# Split dataset into training and testing sets
# تقسیم داده‌ها به آموزش و آزمون
split_index = int(len(df) * 0.8)  # 80% آموزش، 20% آزمون
X_train = x[:split_index]
X_test = x[split_index:]

X_train.shape, X_test.shape

In [ ]:
# تعریف طول دنباله (مثلاً ۱۰ روز)
sequence_length = 10  
# انتخاب اندیس ستون‌هایی که می‌خواهیم پیش‌بینی کنیم: High, Low, Close (اندیس‌های ۱، ۲، ۳)
target_columns = [1, 2, 3]

# تابع ایجاد دنباله‌های زمانی
def create_sequences(data, seq_length, target_cols):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])           # دنباله ورودی به طول seq_length
        y.append(data[i+seq_length, target_cols])  # پیش‌بینی مقادیر روز بعد (High, Low, Close)
    return np.array(X), np.array(y)

# ایجاد دنباله‌های آموزشی و آزمون
X_train_seq, y_train = create_sequences(X_train, sequence_length, target_columns)
X_test_seq, y_test = create_sequences(X_test, sequence_length, target_columns)

print("shape X_train_seq:", X_train_seq.shape)
print("shape y_train:", y_train.shape)
print("shape X_test_seq:", X_test_seq.shape)
print("shape y_test:", y_test.shape)

In [ ]:
# تعریف مدل LSTM بهبود یافته
def create_model(input_shape):
    model = Sequential()
    
    # لایه اول Bidirectional LSTM با تنظیمات بهینه
    model.add(Bidirectional(
        LSTM(128, activation='tanh', return_sequences=True, 
             kernel_regularizer=L2(0.001)),
        input_shape=input_shape
    ))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    
    # لایه دوم LSTM
    model.add(Bidirectional(
        LSTM(64, activation='tanh', return_sequences=True,
            kernel_regularizer=L2(0.001))
    ))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    
    # لایه سوم LSTM
    model.add(Bidirectional(
        LSTM(32, activation='tanh',
            kernel_regularizer=L2(0.001))
    ))
    model.add(Dropout(0.3))
    model.add(BatchNormalization())
    
    # لایه‌های Dense
    model.add(Dense(32, activation='relu', kernel_regularizer=L2(0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(3, activation='linear'))  # یک خروجی با ۳ مقدار
    
    # کامپایل اصلاح شده
    optimizer = Adam(learning_rate=0.0005)
    model.compile(optimizer=optimizer, 
                loss='mse',  # استفاده از یک loss واحد
                metrics=['mae'])
    return model

# تعیین شکل ورودی: (تعداد روزها در دنباله, تعداد ویژگی‌ها)
input_shape = (X_train_seq.shape[1], X_train_seq.shape[2])
model = create_model(input_shape)
model.summary()

In [ ]:
# تعریف Callbackها با تنظیم دقیق‌تر
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

# آموزش مدل
history = model.fit(
    X_train_seq, y_train,
    validation_data=(X_test_seq, y_test),
    epochs=250,             
    batch_size=32,
    callbacks=[early_stopping, lr_scheduler]
)


In [ ]:
# پیش‌بینی مقادیر با داده تست
y_pred_scaled = model.predict(X_test_seq)

# ساخت آرایه کامل با شکل (نمونه‌ها, تمام ویژگی‌ها) برای inverse transform
dummy_array = np.zeros((len(y_test), x.shape[1]))
dummy_array[:, target_columns] = y_pred_scaled  # جایگذاری مقادیر پیش‌بینی شده

# تبدیل به مقیاس واقعی
y_pred_real = scaler.inverse_transform(dummy_array)[:, target_columns]

In [ ]:
start_idx = split_index + sequence_length
end_idx = start_idx + len(y_test)
y_true_real = df.iloc[start_idx:end_idx][['High', 'Low', 'Close']].values

print("نمونه‌ای از پیش‌بینی‌ها در مقیاس اصلی:")
print(y_pred_real[:5])
print("نمونه‌ای از مقادیر واقعی در مقیاس اصلی:")
print(y_true_real[:5])

نمونه‌ای از پیش‌بینی‌ها در مقیاس اصلی:
[[30888.45587446 29509.71347797 30336.34898007]
 [31425.04964103 30017.07494514 30869.0461729 ]
 [31833.7033442  30404.29939991 31276.87281356]
 [32691.24968421 31216.94799676 32132.86865274]
 [33733.67368987 32203.82295132 33170.91204776]]
نمونه‌ای از مقادیر واقعی در مقیاس اصلی:
[[30287.48242188 29481.75195312 29918.41210938]
 [30199.43359375 29720.3125     29993.89648438]
 [34370.4375     30097.828125   33086.234375  ]
 [35150.43359375 32880.76171875 33901.52734375]
 [35133.7578125  33709.109375   34502.8203125 ]]

نمونه‌ای از پیش‌بینی‌ها در مقیاس اصلی:
[[32071.51269801 30491.92073138 31016.92799248]
 [32757.81094301 31133.23061613 31771.20562626]
 [33336.03782752 31673.43638986 32405.82650963]
 [34276.99234526 32551.77664266 33439.60378802]
 [35372.38589025 33573.7719406  34642.90016331]]
نمونه‌ای از مقادیر واقعی در مقیاس اصلی:
[[30287.48242188 29481.75195312 29918.41210938]
 [30199.43359375 29720.3125     29993.89648438]
 [34370.4375     30097.828125   33086.234375  ]
 [35150.43359375 32880.76171875 33901.52734375]
 [35133.7578125  33709.109375   34502.8203125 ]]

In [ ]:
plt.figure(figsize=(14, 7))
for i, col in enumerate(['High', 'Low', 'Close']):
    plt.subplot(3, 1, i+1)
    plt.plot(y_true_real[:, i], label='Real')
    plt.plot(y_pred_real[:, i], label='Predicted')
    plt.title(f'Prediction for {col}')
    plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
metrics = {
    'MAE': mean_absolute_error(y_true_real, y_pred_real),
    'RMSE': np.sqrt(mean_squared_error(y_true_real, y_pred_real)),
    'R²': r2_score(y_true_real, y_pred_real),
    'MAPE': mean_absolute_percentage_error(y_true_real, y_pred_real)
}

print("نتایج ارزیابی:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

نتایج ارزیابی:
MAE: 3657.5138
RMSE: 5016.6410
R²: 0.9291
MAPE: 0.0537

نتایج ارزیابی:
MAE: 3908.0081
RMSE: 5538.2381
R²: 0.9132
MAPE: 0.0559

In [ ]:
# برای هر یک از ستون‌های پیش‌بینی (High, Low, Close)
columns_names = ['High', 'Low', 'Close']

for i, name in enumerate(columns_names):
    mae = mean_absolute_error(y_true_real[:, i], y_pred_real[:, i])
    rmse = np.sqrt(mean_squared_error(y_true_real[:, i], y_pred_real[:, i]))
    r2 = r2_score(y_true_real[:, i], y_pred_real[:, i])
    
    print(f"\nمعیارها برای {name}:")
    print(f"MAE  : {mae:.4f}")
    print(f"RMSE : {rmse:.4f}")
    print(f"R^2  : {r2:.4f}")


In [ ]:
model.save('pred_BTC_ind.h5')